In [14]:
'''goal of this notebook: use pandas to organize the data into training data.'''
import pandas as pd
from bs4 import BeautifulSoup
import os
from datetime import datetime as dt

#set directory and extract file contents in a loop
#datadir = './test_rss' (for test files)
datadir = '../crawl_data/rawdata_archived_rss_files'
datafiles = os.listdir(datadir)
print(datafiles[:20])
max_tr_limit = 1000

### open a file
#f = open(datadir + '/' + '733_rss_20160519')
#print(f.read())

list_dfs = []
for i, fname in enumerate(datafiles):
    if i >= max_tr_limit:
        break
    
    #open an rss file, convert its html elements to beautiful soup, and extract all links and metadata.
    full_fpath = datadir + '/' + fname
    with open(full_fpath,'rb') as f:
        soup = BeautifulSoup(f.read())
        #print(soup)

        #for each element, need to store it in pandas form. {Can I do this any faster???}
        items = soup.findAll("item")
        for item in items:
            #note: was not able to get the link. That will require some extra work.
            df_item = pd.DataFrame({
                'title':[item.title.text.strip()],
                'description':[item.description.text.strip()],
                'date':[item.pubdate.text.strip()[5:16]]
            })

            #aggregate all "item"-dataframes.
            list_dfs.append(df_item)

rawitems_df = pd.concat(list_dfs, axis=0).reset_index(drop=True)
print(rawitems_df.head())
print(rawitems_df.shape)
    

['733_rss_20160519', '5292_rss_20141227', '873_rss_20160527', '1337_rss_20160629', '5846_rss_20141206', '1182_rss_20160620', '6249_rss_20160420', '1074_rss_20160615', '2473_rss_20160408', '3683_rss_20160630', '1582_rss_20160713', '3042_rss_20160521', '2735_rss_20160422', '4536_rss_20160608', '2760_rss_20160422', '2362_rss_20140107', '1850_rss_20160824', '3763_rss_20160705', '6482_rss_20160510', '6662_rss_20160519']
                                               title description         date
0           Traders: Here's how we're playing retail              19 May 2016
1  The Fed brought this fear back to markets this...              19 May 2016
2  What we know about missing EgyptAir Flight MS8...              19 May 2016
3     CIO: Investors in these stocks, beware of this              19 May 2016
4  Theranos CEO is looking for a new executive as...              19 May 2016
(29929, 3)


In [ ]:
'''
    "sql" steps:
    1. for each ticker, pull out all articles related to that ticker.
        i. ~ create new columns in our dataframe to say which tickers belong to which titles.
        ii. also, need to figure out which points have enough data.
    
    2. reorganize the dataframe:
        i. group by ticker and date. When you do this, every group will be a new data point.
        ii. Combine this with the stock movement of the given stock on the given day.
        iii. Now we should be ready for data analysis.
'''


In [15]:
### Q: what is the best way to search within pandas?? 
##  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html
'''
    This cell: convert the html data from raw data to data that is ready for ML:
    Namely, in our input data for ML, we want one row for each ticker and each day.
'''

'''
    Search for tickers within titles, and create a new column for each ticker.
'''
def ticker_label_columns(tickers, rawitems_df):
    l1 = []
    contains_ticker_cols = [rawitems_df]
    for t1 in tickers:
        contains_ti = l1.copy()
        ticker_title = t1.replace(' ','_') #come up with another function, or a dictionary?
        contains_ti = rawitems_df['title'].str.contains(t1, case=False, regex=False).rename(ticker_title)
        contains_ticker_cols.append(contains_ti)
    res1 = pd.concat(contains_ticker_cols, axis=1)
    return res1


'''
    Group by ticker and date. Combine all the titles into a single list of strings type.
    ##! need to test this out on a bigger dataset
'''
def concat_titles_tk_date(res1, tickers):
    ticker_frames = []
    for t1 in tickers:
        ticker_title = t1.replace(' ','_') ## need something better!
        #selects only the rows of "res1" where the ticker name is present (or res[t1]=True)
        t1_cols = res1[res1[ticker_title]].loc[:,['title','date']]
        
        #for all the columns sharing a date, merges the titles into a list. Add cols for date & ticker.
        t1_conc_titles = t1_cols.groupby('date')['title'].apply(list).rename('list_titles')
        t1_lengths = t1_conc_titles.agg(len).rename('length')
        r1 = pd.concat([t1_conc_titles, t1_lengths], axis=1)
        t1_res3 = r1.reset_index()
        t1_res3['ticker'] = t1

        ticker_frames.append(t1_res3)
    res2 = pd.concat(ticker_frames,axis=0).reset_index(drop=True) ##this is ready for data analysis.
    return res2



#------------------------------------------------------------------#
### run all the functions down here

#come up with more tickers
tickers = ['facebook','netflix','apple','google','amazon', 'aramco', 'morgan stanley', 'chase'] 

#create columns for labeling each ticker within title.
res1 = ticker_label_columns(tickers, rawitems_df)
print(res1.head())

#aggregate titles by ticker & date.
print("\n\nres2:")
res2 = concat_titles_tk_date(res1, tickers)
print(res2.head())


### ^^ next, join the above with real stock movement data for the given dates.

#then, save the result as a pickle.

                                               title description         date  \
0           Traders: Here's how we're playing retail              19 May 2016   
1  The Fed brought this fear back to markets this...              19 May 2016   
2  What we know about missing EgyptAir Flight MS8...              19 May 2016   
3     CIO: Investors in these stocks, beware of this              19 May 2016   
4  Theranos CEO is looking for a new executive as...              19 May 2016   

   facebook  netflix  apple  google  amazon  aramco  morgan_stanley  chase  
0     False    False  False   False   False   False           False  False  
1     False    False  False   False   False   False           False  False  
2     False    False  False   False   False   False           False  False  
3     False    False  False   False   False   False           False  False  
4     False    False  False   False   False   False           False  False  


res2:
          date                             

In [16]:
'''
###############################
## debug the list creation...
###############################

# April 8th: too many fb articles...

#result... high #s of duplicates are coming from the data, not from aggregation.
'''
t1_cols = res1[res1['facebook']].loc[:,['title','date']]
r1 = t1_cols[t1_cols.date == '08 Apr 2016']
print(r1.shape)
#print(r1)

## q: is the problem present in res1?
r2 = res1[res1.title == "Facebook taking shady retailers 'very seriously'"]
print(r2.shape)
print(r2)

print("------------------")
print(r1)

#for all the columns sharing a date, merges the titles into a list. Add cols for date & ticker.
#t1_conc_titles = t1_cols.groupby('date')['title'].apply(list).rename('list_titles')

(9, 2)
(2, 11)
                                                 title description  \
1506  Facebook taking shady retailers 'very seriously'               
2853  Facebook taking shady retailers 'very seriously'               

             date  facebook  netflix  apple  google  amazon  aramco  \
1506  08 Apr 2016      True    False  False   False   False   False   
2853  08 Apr 2016      True    False  False   False   False   False   

      morgan_stanley  chase  
1506           False  False  
2853           False  False  
------------------
                                                   title         date
1506    Facebook taking shady retailers 'very seriously'  08 Apr 2016
1523                'Chatbots' may be coming to Facebook  08 Apr 2016
2853    Facebook taking shady retailers 'very seriously'  08 Apr 2016
2870                'Chatbots' may be coming to Facebook  08 Apr 2016
6908                'Chatbots' may be coming to Facebook  08 Apr 2016
8632   Facebook live video set 

In [6]:
'''
    In this cell, pull in the stock market data from csv files
'''
def extract_historical_stocks(stocks_data_dir):
    stocks_files = os.listdir(stocks_data_dir)

    #for each file, make the file into a pandas dataframe.
    df = pd.DataFrame()
    all_tkr_dfs = []
    for sfile in stocks_files:
        r1 = df.copy()

        full_path = stocks_data_dir + '/' + sfile
        stockdf1 = pd.read_csv(full_path)
        sfile_tkr = sfile[:-4]
        # print(stockdf1.head())

        #convert dates to the right format
        dates = stockdf1['Date'].apply(lambda x: dt.strptime(x,'%m/%d/%Y').strftime('%d %b %Y')).rename('date')
        #require a space (' ') before the key name... Also column names: "stockdf.columns"
        #here we convert string stock price to float.
        close_prc = stockdf1[' Close/Last'].apply(lambda x: float(x.strip()[1:])).rename('Close')
        open_prc = stockdf1[' Open'].apply(lambda x: float(x.strip()[1:])).rename('Open')

        r1 = pd.concat([dates, open_prc, close_prc], axis=1)
        r1['ticker'] = sfile_tkr
        all_tkr_dfs.append(r1)
        #print(r1)

    #combine everything at the end.
    historical_stock_prices = pd.concat(all_tkr_dfs, axis=0).reset_index(drop=True)
    return historical_stock_prices


stocks_data_dir = '../crawl_data/stock_data'
historical_stock_prices = extract_historical_stocks(stocks_data_dir)
print(historical_stock_prices[historical_stock_prices.ticker=='facebook'].head())

             date    Open   Close    ticker
1258  16 Dec 2019  195.27  197.92  facebook
1259  13 Dec 2019  196.40  194.11  facebook
1260  12 Dec 2019  202.35  196.75  facebook
1261  11 Dec 2019  200.28  202.26  facebook
1262  10 Dec 2019  201.66  200.87  facebook


In [17]:
'''
    Utilize joins to say which days the stock price dropped and when it gained. Once this is combined with the
    features, we will have the full "trdata" database created.
    
    TODO: Run all the above code for all files (once crawled) and save to a pickle.
'''
#df1: historical_stock_prices; df2: "res2" - converted the raw data to "training" format.
r1 = historical_stock_prices
trdata = res2.join(r1.set_index(['date','ticker']), on=['date', 'ticker'])
print(trdata.head())

### now we finally have the final form of "trdata"!!! 

          date                                        list_titles  length  \
0  01 Jul 2016  [Ex-Facebooker reveals how the ‘cult’ of Silic...      13   
1  02 Jun 2016  [Gawker founder says Facebook should be held a...       1   
2  03 Aug 2016  [Facebook squares off against Twitter, Snapcha...       8   
3  03 Jun 2016  [Facebook board seeks curb in Zuckerberg contr...       2   
4  04 Sep 2015  [Five details you shouldn't give Facebook, Fac...       2   

     ticker     Open   Close  
0  facebook  114.200  114.19  
1  facebook  118.690  118.93  
2  facebook  123.090  122.51  
3  facebook  118.975  118.47  
4  facebook   87.200   88.26  


In [8]:
## store the result from above as a pickle.
import pickle as pkl
with open('./pandas_trdata/trdata.pickle','wb') as f:
    pkl.dump(trdata, f)

## Below, explore the full Training Data just created.

In [18]:
## get some stats for the data we just collected...
print('fb: ',end='')
print(sum(res1['facebook']))

print('netflix: ',end='')
print(sum(res1['netflix']))

print('google: ',end='')
print(sum(res1['google']))

print('apple: ',end='')
print(sum(res1['apple']))

print('amzn: ',end='')
print(sum(res1['amazon']))

## more data analysis...
print("")
print(res1.shape)

print(res2.shape)

### problem: this is way too little training data to train our lstm. How can we get more?
### we will have to collect at least 2-3k date/ticker combos total. 

'''
    problem: 2016 has 433 date-ticker pairs. In order to train a satisfactory LSTM, we need
    around 10k data points (best case scenario). A minimum threshold would be around 2-3k examples...
    
    solution? 
    1. Think about looking into some low-data approaches?
    2. data augmentation?
        ~ Can we change it so it's no longer a list? We would need to eliminate dupes...
    3. try it out anyway
    4. need some other way to collect news headlines.
    
    
    Here are some other sources for news data:
    0. A kaggle competition was done on this exact topic: https://www.kaggle.com/c/two-sigma-financial-news/rules
    1. reuters: https://www.reutersagency.com/en/contact/
    2. A dataset of full articles!: https://www.kaggle.com/snapcrack/all-the-news
'''

fb: 206
netflix: 100
google: 378
apple: 515
amzn: 207

(29929, 11)
(433, 4)


In [125]:
#res2

t1_cols = res1[res1['facebook']].loc[:,['title','date']]
r2 = t1_cols.groupby('date')['title'].apply(list).rename('list_titles')
#print(r2)
#print('#--------------------------------------#')
r3 = r2.agg(len).rename('length')

r4 = pd.concat([r2, r3], axis=1)

print(r4)

# [this got the length...] t1_cols.groupby('date')['title'].apply(list).agg(len)

                                                   list_titles  length
date                                                                  
01 Jul 2016  [Ex-Facebooker reveals how the ‘cult’ of Silic...       3
02 Jun 2016  [Gawker founder says Facebook should be held a...       1
03 Jun 2016  [Facebook board seeks curb in Zuckerberg contr...       1
05 Jul 2016  [Twitter adds ex-Facebook technology chief to ...       1
06 May 2016  [Facebook's 'screen queen' leaving Oculus, Fac...       2
07 Apr 2016  [The firm helping Facebook get $$$ in lost rev...       2
07 Jul 2016           [DOJ sues to enforce Facebook tax probe]       1
08 Apr 2016  [Facebook taking shady retailers 'very serious...       4
11 Jun 2016  ['Moments' in time: Facebook tells users to do...       2
13 Jul 2016  [Inside the secret lab where Facebook tries to...       1
13 May 2016  [Zuckerberg says Facebook probing claims ‘tren...       1
16 Jun 2016  [Short Apple, long Samsung and Facebook could ...       1
18 Apr

In [69]:

#res1.groupby(['apple','date']).filter('apple')

#date: \n Wed, 23 Jan 2013 22:42 GMT\n
res2 = res1[res1['apple']].loc[:,['title','date']]
print(res2)

print('#--------------------------------------------#')
for name, group in res2.groupby('date'):
    print(name)
    print(group)

    
    
#the below is pretty close. Figure out how to add the ticker back into it.
print('#--------------------------------------------#')
r3 = res2.groupby('date')['title'].apply(list)
res3 = r3.reset_index()
res3['ticker']='apple'
    ## come up with one of these per group, and we will be done.
print(res3)

                                                title  \
2   \n      Apple's Revenue Falls Short, Shares Di...   
23  \n      Jobs Threatened Suit to Prevent Apple ...   

                                         date  
2   \n      Wed, 23 Jan 2013 22:42 GMT\n       
23  \n      Wed, 23 Jan 2013 13:48 GMT\n       
#--------------------------------------------#

      Wed, 23 Jan 2013 13:48 GMT
     
                                                title  \
23  \n      Jobs Threatened Suit to Prevent Apple ...   

                                         date  
23  \n      Wed, 23 Jan 2013 13:48 GMT\n       

      Wed, 23 Jan 2013 22:42 GMT
     
                                               title  \
2  \n      Apple's Revenue Falls Short, Shares Di...   

                                        date  
2  \n      Wed, 23 Jan 2013 22:42 GMT\n       
#--------------------------------------------#
                                        date  \
0  \n      Wed, 23 Jan 2013 13:48 GMT\n     

In [71]:
### do some data cleaning
a = 5
print(a)

5


In [13]:
## check out the earlier pickle that I created...
### 1446 headlines... I don't know what happened to the 2018 and 2019 data...
import pickle as pkl
unpickle = False
if unpickle:
    fdir = '../crawl_data/rss_feed_articles.pickle'
    with open(fdir, 'rb') as f:
        data = pkl.load(f)
    
print(len(data))

1446
